In [1]:
!pip install fire
!pip install torch
!pip install datasets
!pip install python-dotenv
!pip install peft
!pip uninstall transformers -y
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116935 sha256=843fe652f487c3ff691f8df7cf427c832caf3855baab34033196c03b9531e040
  Stored in directory: /root/.cache/pip/wheels/34/a9/61/d515d3cd1e8a349fed305bc67a9c7d68fc38d51053b6decad6
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 110.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 31.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 103.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 19.4 MB/s eta 0:0

In [3]:
import os
import sys
import warnings
from dataclasses import dataclass, field
from typing import List, Optional, Tuple

import fire
import torch
from datasets import load_dataset
from dotenv import load_dotenv
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    # prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq,
    PreTrainedModel,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments
)
from transformers.tokenization_utils_base import logger as tokenization_logger

from utils.prompter import Prompter

In [5]:
load_dotenv()
warnings.filterwarnings(
    "ignore",
    message=".*GPTNeoXTokenizerFast.*",
    category=UserWarning,
    module="transformers.tokenization_utils_base",
)
tokenization_logger.setLevel("ERROR")
torch.cuda.empty_cache()


@dataclass
class TrainConfig:
    base_model: str
    data_path: str = "yahma/alpaca-cleaned"
    output_dir: str = "./lora-alpaca"
    device_map: str = "auto"
    batch_size: int = 128
    micro_batch_size: int = 4
    num_epochs: int = 3
    learning_rate: float = 3e-4
    cutoff_len: int = 256
    val_set_size: int = 12
    lora_r: int = 8
    lora_alpha: int = 16
    lora_dropout: float = 0.05
    lora_target_modules: List[str] = field(
        default_factory=lambda: ["query_key_value"]
    )
    train_on_inputs: bool = True
    add_eos_token: bool = False
    group_by_length: bool = False
    resume_from_checkpoint: Optional[str] = None
    prompt_template_name: str = "alpaca"


class TokenizerHelper:
    def __init__(
        self, prompter, tokenizer, train_on_inputs, cutoff_len, add_eos_token=True
    ):
        self.prompter = prompter
        self.tokenizer = tokenizer
        self.train_on_inputs = train_on_inputs
        self.add_eos_token = add_eos_token
        self.cutoff_len = cutoff_len

    def tokenize(self, prompt):
        result = self.tokenizer(
            prompt,
            truncation=True,
            max_length=self.cutoff_len,
            # Set padding to 'max_length' instead of False for GPTNeoXTokenizerFast???
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != self.tokenizer.eos_token_id
            and len(result["input_ids"]) < self.cutoff_len
            and self.add_eos_token
        ):
            result["input_ids"].append(self.tokenizer.eos_token_id)
            result["attention_mask"].append(1)
        return result

    def generate_and_tokenize_prompt(self, data_point):
        full_prompt = self.prompter.generate_prompt(
            data_point["instruction"],
            data_point["input"],
            data_point["output"],
        )
        tokenized_full_prompt = self.tokenize(full_prompt)

        if not self.train_on_inputs:
            user_prompt = self.prompter.generate_prompt(
                data_point["instruction"], data_point["input"]
            )
            tokenized_user_prompt = self.tokenize(user_prompt)
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            if self.add_eos_token:
                user_prompt_len -= 1

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["input_ids"][
                user_prompt_len:
            ]  # could be sped up, probably
        else:
            tokenized_full_prompt["labels"] = tokenized_full_prompt["input_ids"]

        return tokenized_full_prompt

def setup_model(config: TrainConfig) -> Tuple[PreTrainedModel, PreTrainedTokenizer]:
    quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)
    model = AutoModelForCausalLM.from_pretrained(
        config.base_model,
        trust_remote_code=True,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=config.device_map,
        quantization_config=quantization_config,
    )
    tokenizer = AutoTokenizer.from_pretrained(config.base_model)

    # LoRA configuration
    lora_config = LoraConfig(
        r=config.lora_r,
        lora_alpha=config.lora_alpha,
        target_modules=config.lora_target_modules,
        lora_dropout=config.lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, lora_config)

    return model, tokenizer


def load_data(config: TrainConfig) -> Tuple:
    """TODO: Not working yet.

    Args:
        config (TrainConfig): _description_

    Returns:
        Tuple: _description_
    """
    # Load the dataset
    dataset = load_dataset(config.dataset_name)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, mlm=False)

    # Split the dataset into train, validation and (optionally) test sets
    train_dataset = tokenized_dataset["train"]
    val_dataset = tokenized_dataset["validation"]

    if "test" in tokenized_dataset:
        test_dataset = tokenized_dataset["test"]
    else:
        test_dataset = None

    return train_dataset, val_dataset, test_dataset, data_collator


def train(
    # model/data params
    base_model: str = "togethercomputer/RedPajama-INCITE-Base-7B-v0.1",  # the only required argument
    data_path: str = "Finetune-test-2.json",
    output_dir: str = "./lora-alpaca",
    # training hyperparams
    batch_size: int = 128,
    micro_batch_size: int = 4,
    num_epochs: int = 3,
    learning_rate: float = 3e-4,
    cutoff_len: int = 256,
    val_set_size: int = 12,
    # lora hyperparams
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    lora_target_modules: List[str] = ["query_key_value", "xxx"],
    train_on_inputs: bool = True,  # if False, masks out inputs in loss
    add_eos_token: bool = False,
    group_by_length: bool = False,  # faster, but produces an odd training loss curve
    resume_from_checkpoint: str = None,  # either training checkpoint or final adapter
    prompt_template_name: str = "alpaca",  # Prompt template to use, default to Alpaca
):
    if int(os.environ.get("LOCAL_RANK", 0)) == 0:
        print(
            f"\n\n\nLoRA fine-tuning model with params:\n"
            f"base_model: {base_model}\n"
            f"data_path: {data_path}\n"
            f"output_dir: {output_dir}\n"
            f"batch_size: {batch_size}\n"
            f"micro_batch_size: {micro_batch_size}\n"
            f"num_epochs: {num_epochs}\n"
            f"learning_rate: {learning_rate}\n"
            f"cutoff_len: {cutoff_len}\n"
            f"val_set_size: {val_set_size}\n"
            f"lora_r: {lora_r}\n"
            f"lora_alpha: {lora_alpha}\n"
            f"lora_dropout: {lora_dropout}\n"
            f"lora_target_modules: {lora_target_modules}\n"
            f"train_on_inputs: {train_on_inputs}\n"
            f"add_eos_token: {add_eos_token}\n"
            f"group_by_length: {group_by_length}\n"
            f"resume_from_checkpoint: {resume_from_checkpoint or False}\n"
            f"prompt template: {prompt_template_name}\n"
        )
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='huggyllama/llama-7b'"

    gradient_accumulation_steps = batch_size // micro_batch_size

    prompter = Prompter(prompt_template_name)

    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        gradient_accumulation_steps = gradient_accumulation_steps // world_size
    print(f"device map: {device_map}")

    #
    # Model loading
    #
    quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)
    model = AutoModelForCausalLM.from_pretrained(
        # 'mosaicml/mpt-7b',
        base_model,
        trust_remote_code=True,
        # base_model,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
        quantization_config=quantization_config,
        # load_in_8bit_fp32_cpu_offload=True
    )

    tokenizer = AutoTokenizer.from_pretrained(base_model)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
    tokenizer.padding_side = "left"  # Allow batched inference

    #
    # 8-bit training
    #
    # had to turn int8 training off for some reason. could it be the titan rtx?
    # turned it on and kinda working now, but wtf?
    # model = prepare_model_for_int8_training(model)

    #
    # LoRA
    #
    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

    if data_path.endswith(".json") or data_path.endswith(".jsonl"):
        data = load_dataset("json", data_files=data_path)
    else:
        data = load_dataset("text", data_files=data_path)

    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(
                resume_from_checkpoint, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = False  # So the trainer won't try loading its state
        # The two files above have a different name depending on how they were saved,
        # but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")

    # Be more transparent about the % of trainable params.
    model.print_trainable_parameters()

    tokenizer_helper = TokenizerHelper(
        prompter, tokenizer, train_on_inputs, cutoff_len, add_eos_token
    )

    if val_set_size > 0:
        train_val = data["train"].train_test_split(
            test_size=val_set_size, shuffle=True, seed=42
        )
        train_data = (
            train_val["train"]
            .shuffle()
            .map(tokenizer_helper.generate_and_tokenize_prompt)
        )
        val_data = (
            train_val["test"]
            .shuffle()
            .map(tokenizer_helper.generate_and_tokenize_prompt)
        )
    else:
        train_data = (
            data["train"].shuffle().map(tokenizer_helper.generate_and_tokenize_prompt)
        )
        val_data = None

    if not ddp and torch.cuda.device_count() > 1:
        # keeps Trainer from trying its own DataParallelism
        # when more than 1 gpu is available
        model.is_parallelizable = True
        model.model_parallel = True

    use_wandb = False
    trainer = Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=TrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=100,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=200 if val_set_size > 0 else None,
            save_steps=200,
            output_dir=output_dir,
            save_total_limit=3,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            group_by_length=group_by_length,
            report_to="wandb" if use_wandb else None,
            run_name=wandb_run_name if use_wandb else None,
        ),
        data_collator=DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )
    model.config.use_cache = False

    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
    ).__get__(model, type(model))

    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    model.save_pretrained(output_dir)

    print("\n If there's a warning about missing keys above, please disregard :)")


if __name__ == "__main__":
    fire.Fire(train)




LoRA fine-tuning model with params:
base_model: togethercomputer/RedPajama-INCITE-Base-7B-v0.1
data_path: Finetune-test-2.json
output_dir: ./lora-alpaca
batch_size: 128
micro_batch_size: 4
num_epochs: 3
learning_rate: 0.0003
cutoff_len: 256
val_set_size: 12
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['query_key_value', 'xxx']
train_on_inputs: True
add_eos_token: False
group_by_length: False
resume_from_checkpoint: False
prompt template: alpaca

device map: auto


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using custom data configuration default-548c3061a4bc1c3b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

...

Step,Training Loss,Validation Loss



 If there's a warning about missing keys above, please disregard :)


ERROR: Could not consume arg: -f
Usage: ipykernel_launcher.py -

For detailed information on this command, run:
  ipykernel_launcher.py - --help


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3378 in run_code         │
│                                                                                                  │
│   3375 │   │                                                                                     │
│   3376 │   │   compiler : callable                                                               │
│   3377 │   │     A function with the same interface as the built-in compile(), to turn           │
│ ❱ 3378 │   │     the AST nodes into code objects. Default is the built-in compile().             │
│   3379 │   │   result : ExecutionResult, optional                                                │
│   3380 │   │     An object to store exceptions that occur during execution.                      │
│   3381                                                                                           │
│                                                                                                  │
│ in <module>                                                                                      │
│                                                                                                  │
│   352                                                                                            │
│   353                                                                                            │
│   354 if __name__ == "__main__":                                                                 │
│ ❱ 355 │   fire.Fire(train)                                                                       │
│   356                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/fire/core.py:145 in Fire                                  │
│                                                                                                  │
│   142                                                                                            │
│   143   if component_trace.HasError():                                                           │
│   144 │   _DisplayError(component_trace)                                                         │
│ ❱ 145 │   raise FireExit(2, component_trace)                                                     │
│   146   if component_trace.show_trace and component_trace.show_help:                             │
│   147 │   output = ['Fire trace:\n{trace}\n'.format(trace=component_trace)]                      │
│   148 │   result = component_trace.GetResult()                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FireExit: 2

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  """
